In [1]:
import pandas as pd
import os

import requests
import re
import time
import numpy as np
import subprocess
import random

from bs4 import BeautifulSoup

### gombis.pl

In [12]:
page_no = 1
raw_data = []

URL = "https://gombis.pl/"
response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')

#retrieve categories
category_container = soup.find('ul', class_='activelinks')
categories = [x.text for x in category_container.find_all('li')]
cat_urls = [x.find('a')['href'] for x in category_container.find_all('li')]

for cat, cat_url in zip(categories, cat_urls):
    print(cat,cat_url)
    
raw_data = []


#retrieve full game list
for cat, cat_url in zip(categories, cat_urls):
    page_no = 1
    while True:
        url = cat_url + '/' + str(page_no)
        print(url)
        response = requests.get(url)
        if response.url != url and page_no>1:
            #done,no more contents
            break
        
        #parse page results
        soup = BeautifulSoup(response.text, 'html.parser')
        game_containers = soup.find_all('div', class_='thumb-game thumb-game_l')
        for game in game_containers:
            raw_data.append({'game':game.find('a').text, 
                             'category' : cat, 
                             'url':game.find('a')['href']})
#             print (game.find('a').text, game.find('a')['href'])
        page_no+=1        

 2 Osoby https://gombis.pl/dwie-osoby
 Dziewczyn https://gombis.pl/dla-dziewczyn
 samochodowe https://gombis.pl/samochodowe
 Akcji https://gombis.pl/akcji
 przygodowe https://gombis.pl/przygodowe
 .io https://gombis.pl/io
 Puzzle https://gombis.pl/zagadki-logiczne
 Sportowe https://gombis.pl/sportowe
https://gombis.pl/dwie-osoby/1
https://gombis.pl/dwie-osoby/2
https://gombis.pl/dwie-osoby/3
https://gombis.pl/dla-dziewczyn/1
https://gombis.pl/dla-dziewczyn/2
https://gombis.pl/dla-dziewczyn/3
https://gombis.pl/dla-dziewczyn/4
https://gombis.pl/dla-dziewczyn/5
https://gombis.pl/dla-dziewczyn/6
https://gombis.pl/dla-dziewczyn/7
https://gombis.pl/dla-dziewczyn/8
https://gombis.pl/dla-dziewczyn/9
https://gombis.pl/dla-dziewczyn/10
https://gombis.pl/dla-dziewczyn/11
https://gombis.pl/dla-dziewczyn/12
https://gombis.pl/dla-dziewczyn/13
https://gombis.pl/samochodowe/1
https://gombis.pl/samochodowe/2
https://gombis.pl/samochodowe/3
https://gombis.pl/samochodowe/4
https://gombis.pl/samochodowe/5

In [13]:
df_games = pd.DataFrame(raw_data)
df_games

,game,category,url
0,Zombie Parade Defense,2 Osoby,https://gombis.pl/g/zombie-parade-defense
1,Desert City Stunt,2 Osoby,https://gombis.pl/g/desert-city-stunt
2,Heads Mayhem,2 Osoby,https://gombis.pl/g/heads-mayhem
3,City Car Stunt 3,2 Osoby,https://gombis.pl/g/city-car-stunt-3
4,Drunken Duel,2 Osoby,https://gombis.pl/g/drunken-duel
...,...,...,...
1802,Bilard multiplayer online,Sportowe,https://gombis.pl/g/bilard-multiplayer-online
1803,Flip Master,Sportowe,https://gombis.pl/g/flip-master
1804,Mistrz narciarstwa alpejskiego,Sportowe,https://gombis.pl/g/mistrz-narciarstwa-alpejsk...
1805,Stempel mistrzostwa boksu,Sportowe,https://gombis.pl/g/stempel-mistrzostwa-boksu


In [21]:
#retrieve game metadata
raw_data = []
for i, row in df_games.iterrows():
    response = requests.get(row['url'])
    soup = BeautifulSoup(response.text, 'html.parser')
    likes = int(soup.find('div', class_='likes').text)
    dislikes = int(soup.find('div', class_='dislikes').text)
    
    desc = soup.select('div[class*="GameTags"]')[0].text.split('\n')[2]
    raw_data.append({'game':row['game'], 
                     'url':row['url'], 
                     'category':row['category'], 
                     'likes':likes,
                     'dislikes':dislikes, 
                     'desc':desc})
df_info = pd.DataFrame(raw_data)
df_info.to_csv("gombis_game_info.csv")

In [22]:
df_info

,game,url,category,likes,dislikes,desc
0,Zombie Parade Defense,https://gombis.pl/g/zombie-parade-defense,2 Osoby,6,1,Gry Akcji Gry Zombie Zombie Parade Defe...
1,Desert City Stunt,https://gombis.pl/g/desert-city-stunt,2 Osoby,14,3,Gry Wyścigowe Gry samochodowe Desert Ci...
2,Heads Mayhem,https://gombis.pl/g/heads-mayhem,2 Osoby,13,2,Gry Akcji Gry Strzelanki Heads Mayhem P...
3,City Car Stunt 3,https://gombis.pl/g/city-car-stunt-3,2 Osoby,26,7,Gry Wyścigowe Gry samochodowe City Car ...
4,Drunken Duel,https://gombis.pl/g/drunken-duel,2 Osoby,21,6,Gry Akcji Gry Stickman Drunken Duel Dru...
...,...,...,...,...,...,...
1802,Bilard multiplayer online,https://gombis.pl/g/bilard-multiplayer-online,Sportowe,15,4,Gry Sportowe Gry Bilard Bilard multipla...
1803,Flip Master,https://gombis.pl/g/flip-master,Sportowe,15,7,Gry Zręcznościowe Gry Skoki Flip Master...
1804,Mistrz narciarstwa alpejskiego,https://gombis.pl/g/mistrz-narciarstwa-alpejsk...,Sportowe,7,0,Gry Sportowe Gry Narciarskie Mistrz nar...
1805,Stempel mistrzostwa boksu,https://gombis.pl/g/stempel-mistrzostwa-boksu,Sportowe,43,10,Gry Sportowe Gry Boks Stempel mistrzost...


### spiels.at

In [37]:
page_no = 1
raw_data = []

URL = "https://spiels.at/kategorien/"
response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')

#retrieve categories
categories = []
cat_urls = []

category_containers = soup.find_all('div', class_='content')
for container in category_containers:
    categories.extend( [x.find('a')['title'] for x in container.find_all('li')] )
    cat_urls.extend( [x.find('a')['href'] for x in container.find_all('li')] )

for cat, cat_url in zip(categories, cat_urls):
    print(cat,cat_url)


.io Spiele https://spiels.at/io-spiele/
10 bis 11 JÃ¤hrige https://spiels.at/10-bis-11-jahrige-spiele/
10 x 10 Puzzle https://spiels.at/10-x-10-puzzle-spiele/
11 bis 12 JÃ¤hrige https://spiels.at/11-bis-12-jahrige-spiele/
3D-Spiele https://spiels.at/3d-spiele/
4 bis 5 JÃ¤hrigen https://spiels.at/4-bis-5-jahrigen-spiele/
4 in einer Reihe https://spiels.at/4-in-einer-reihe-spiele/
5 bis 6 JÃ¤rigen https://spiels.at/5-bis-6-jarigen-spiele/
6 bis 7 JÃ¤hrigen https://spiels.at/6-bis-7-jahrigen-spiele/
7 bis 8 JÃ¤hrige https://spiels.at/7-bis-8-jahrige-spiele/
8 bis 9 JÃ¤hrige https://spiels.at/8-bis-9-jahrige-spiele/
9 bis 10 JÃ¤hrige https://spiels.at/9-bis-10-jahrige-spiele/
Abenteuer https://spiels.at/abenteuer-spiele/
Accessories Make Up https://spiels.at/accessories-make-up-spiele/
Action https://spiels.at/aktion-spiele/
Action-MMO https://spiels.at/action-mmo/
Adam und Eva https://spiels.at/adam-und-eva-spiele/
Air Hockey https://spiels.at/air-hockey-spiele/
Amercian Football-Spiele h

In [38]:
raw_data = []

for cat, cat_url in zip(categories, cat_urls):

    response = requests.get(cat_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    game_containers = soup.find_all('div', class_='game')
#     print(game_containers)
    for game in game_containers:
        try:
            gameurl = game.find('a')['href']
            gamename = game.find('a')['title']
        except:
            continue
        raw_data.append({'game':gamename,
                         'gameurl':gameurl, 
                         'category': cat})

df_games = pd.DataFrame(raw_data)
df_games

,game,gameurl,category
0,Hole.io,/holeio-spiel/,.io Spiele
1,Zombeat.io,/zombeatio-spiel/,.io Spiele
2,AquaPark.io,/aquaparkio-spiel/,.io Spiele
3,SuperHero.io,/superheroio-spiel/,.io Spiele
4,Drawthis.io,/drawthisio-spiel/,.io Spiele
...,...,...,...
5356,Delicious: Emily's Home Sweet Home,/delicious-emily-s-home-sweet-home-spiel/,zylom
5357,Mahjongg Fortuna Basic,/mahjongg-fortuna-basic-spiel/,zylom
5358,Peggle,/peggle-spiel/,zylom
5359,Jewel Quest,/jewel-quest-spiel/,zylom


In [ ]:
raw_data = []


for i, row in df_games.iterrows():
    if i%200 == 0:
        print("Done", i)
    gameurl = "https://spiels.at" + row['gameurl']
#     print(gameurl)
    response = requests.get(gameurl)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    try:
        rating_value = float(soup.find('span', class_='rating-value').text)
        rating_count = float(soup.find('span', class_='rating-count').text.split("Sti")[0])
    except:
        print("Error", gameurl)
        continue
    
    raw_data.append({'game':row['game'],
                     'gameurl':"https://spiels.at" + row['gameurl'], 
                     'rating':rating_value, 
                     'rating_count':rating_count})
    
df_game_info = pd.DataFrame(raw_data)
df_game_info.to_csv("spiels_game_info.csv")

Done 0
Done 200
Done 400
Done 600
Done 800
Done 1000
Error https://spiels.at/royal-vegas-solitaire-spiel/
Done 1200
Done 1400
Done 1600
Done 1800
Error https://spiels.at/royal-vegas-solitaire-spiel/
Done 2000
Done 2200
Done 2400
Error https://spiels.at/royal-vegas-solitaire-spiel/
Done 2600
Error https://spiels.at/dutch-mahjong-spiel/
Done 2800
Done 3000
Done 3200
Done 3400


### ioxgames.com

In [53]:
#get categories
URL = "http://ioxgames.com"
response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')
cat_urls = [x['href'] for x in soup.find_all("a", class_='nav')][0:-1]
categories = [x.text for x in soup.find_all("a", class_='nav')][0:-1]

for x,y in zip(categories, cat_urls):
    print(x,y)

Action http://ioxgames.com/action
Driving http://ioxgames.com/driving
Sports http://ioxgames.com/sports
Style http://ioxgames.com/style
Classic http://ioxgames.com/classic
RPG http://ioxgames.com/rpg
Space http://ioxgames.com/space
Maze http://ioxgames.com/maze
Ocean http://ioxgames.com/ocean
Multiplayer http://ioxgames.com/multiplayer
3D http://ioxgames.com/3d
Puzzle http://ioxgames.com/puzzle
Adventure http://ioxgames.com/adventure


In [57]:
raw_data = []
for cat,cat_url in zip(categories, cat_urls):
    print(cat, cat_url)
    response = requests.get(cat_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    game_containers = soup.find_all('a', class_="item draw")
    for container in game_containers:
        raw_data.append({'game':container.find('span', class_='name').text, 
                         'url':container['href'], 
                         'category':cat})
df = pd.DataFrame(raw_data)
df

Action http://ioxgames.com/action
Driving http://ioxgames.com/driving
Sports http://ioxgames.com/sports
Style http://ioxgames.com/style
Classic http://ioxgames.com/classic
RPG http://ioxgames.com/rpg
Space http://ioxgames.com/space
Maze http://ioxgames.com/maze
Ocean http://ioxgames.com/ocean
Multiplayer http://ioxgames.com/multiplayer
3D http://ioxgames.com/3d
Puzzle http://ioxgames.com/puzzle
Adventure http://ioxgames.com/adventure


,game,url,category
0,ShellShockers.io,http://ioxgames.com/shellshockersio.html,Action
1,ZombsRoyale.io,http://ioxgames.com/zombsroyaleio.html,Action
2,Warbrokers.io,http://ioxgames.com/warbrokersio.html,Action
3,EvoWars.io,http://ioxgames.com/evowarsio.html,Action
4,ZOMBS.io,http://ioxgames.com/zombsio.html,Action
...,...,...,...
101,Jellies.io,http://ioxgames.com/jelliesio.html,Multiplayer
102,Slimes.io,http://ioxgames.com/slimesio.html,Multiplayer
103,Zlax.io,http://ioxgames.com/zlaxio.html,Multiplayer
104,Biome3d.com,http://ioxgames.com/biome3dcom.html,3D


In [70]:
raw_data =[]
for i, row in df.iterrows():
    print(row['url'])
    response = requests.get(row['url'])
    soup = BeautifulSoup(response.text, 'html.parser')
    infos = soup.find_all('table')[1].find_all('th') 
    

    desc = infos[3].text
    uploaded = infos[5].text.lstrip().rstrip()
    playcount = int(infos[7].text.lstrip().rstrip().split(' ')[0].replace(',',''))
#     likes = soup.find(text=re.compile("people like this"))
#     if likes!=None:
#         likes = int(likes.text.split(' ')[0])
#     print(likes)
    raw_data.append({'game':row['game'], 
                     'url':row['url'], 
                     'category':row['category'], 
                     'desc':desc, 
                     'playcount':playcount, 
                     'uploaded':uploaded})
pd.DataFrame(raw_data).to_csv("ioxgames_game_info.csv")

http://ioxgames.com/shellshockersio.html
http://ioxgames.com/zombsroyaleio.html
http://ioxgames.com/warbrokersio.html
http://ioxgames.com/evowarsio.html
http://ioxgames.com/zombsio.html
http://ioxgames.com/warscrapio.html
http://ioxgames.com/diepio.html
http://ioxgames.com/bombomio.html
http://ioxgames.com/slayone.html
http://ioxgames.com/skribblio.html
http://ioxgames.com/zombiecraftio.html
http://ioxgames.com/foesio.html
http://ioxgames.com/racegameio.html
http://ioxgames.com/nitrodriftio.html
http://ioxgames.com/shootemio.html
http://ioxgames.com/hexonzone.html
http://ioxgames.com/deathcario.html
http://ioxgames.com/rollzio.html
http://ioxgames.com/carboomio.html
http://ioxgames.com/cyclewarsio.html
http://ioxgames.com/kartwarsio.html
http://ioxgames.com/driftkingio.html
http://ioxgames.com/nitroclashio.html
http://ioxgames.com/bumpyballio.html
http://ioxgames.com/superballsio.html
http://ioxgames.com/rugbyio.html
http://ioxgames.com/soccerballio.html
http://ioxgames.com/bangoio.htm

In [71]:
pd.DataFrame(raw_data)

,game,url,category,desc,playcount,uploaded
0,ShellShockers.io,http://ioxgames.com/shellshockersio.html,Action,Shellshock.io is a unique .io game in which yo...,4493,Mar 11 2018
1,ZombsRoyale.io,http://ioxgames.com/zombsroyaleio.html,Action,ZombsRoyale.io is a io game of the style Battl...,2583,Apr 23 2018
2,Warbrokers.io,http://ioxgames.com/warbrokersio.html,Action,Battle versus blocky foes in Warbrokers.io! Th...,2055,Mar 11 2018
3,EvoWars.io,http://ioxgames.com/evowarsio.html,Action,Each game starts as a simple cave man. In each...,1594,Mar 10 2018
4,ZOMBS.io,http://ioxgames.com/zombsio.html,Action,ZOMBS.io is Minecraft + Zombie tower defense. ...,1446,Mar 6 2018
...,...,...,...,...,...,...
101,Jellies.io,http://ioxgames.com/jelliesio.html,Multiplayer,"In Jellies.io, dive into the deep ocean and pl...",234,Feb 6 2020
102,Slimes.io,http://ioxgames.com/slimesio.html,Multiplayer,Select your color and get on the canvas of col...,220,Feb 6 2020
103,Zlax.io,http://ioxgames.com/zlaxio.html,Multiplayer,Throw your axes like a real axeman to your opp...,209,Feb 6 2020
104,Biome3d.com,http://ioxgames.com/biome3dcom.html,3D,Biome 3d is essentially a 3d version of agar.i...,855,Mar 5 2018


### game24.ru

In [74]:
response = requests.get("http://en.game24.ru")
soup = BeautifulSoup(response.text, 'html.parser')
categories = ["http://en.game24.ru/" + x['href'] for x in soup.find_all('a', class_='catlist')]
cat_urls = [x['title'].split(" -")[0] for x in soup.find_all('a', class_='catlist')]

http://en.game24.ru/forboys/ Boy Games Online
http://en.game24.ru/forgirls/ Games Online
http://en.game24.ru/forkids/ Games for kids online
http://en.game24.ru/race/ Racing Games Online
http://en.game24.ru/shoot/ Shooting games online
http://en.game24.ru/fighting/ Fighting games online
http://en.game24.ru/adventure/ Adventure Games Online
http://en.game24.ru/fly/ Flying games online
http://en.game24.ru/bubbleshooter/ Balls online game
http://en.game24.ru/logic/ Puzzle games online
http://en.game24.ru/gaming/ Online games
http://en.game24.ru/gamesforadults/ Adult games
http://en.game24.ru/sport/ Sports games online
http://en.game24.ru/quests/ Quest games online
http://en.game24.ru/funny/ Funny games online
http://en.game24.ru/others/ Other online games


In [ ]:
for cat_url,cat in zip(categories, cat_urls):
    processed = set()
    page_no = 1
    while True:
        print(cat, cat_url + "page"+str(page_no), page_no)
        response = requests.get(cat_url + "page"+str(page_no))
        soup = BeautifulSoup(response.text, 'html.parser')
        game_containers = soup.find_all('a', class_="gameslistnew")
        curr_games = set()
        
        for container in game_containers:
            game = container['title']
            url = "http://en.game24.ru" + '/' + container['href']
            curr_games.add(game)
            raw_data.append({'game':game, 'url':url})
            
        #no new games
        if curr_games - processed==set():
            break
        
        processed = processed | curr_games
        page_no +=1
df_games = pd.DataFrame(raw_data)
df_games

In [83]:
df_games = df_games[df_games['url'].str.contains("en.game24.ru")][['game', 'url']]
df_games

,game,url
106,Game online,http://en.game24.ru/6733/
107,Game Fireboy and Watergirl 2 The Light Temple ...,http://en.game24.ru/6729/
108,Game online,http://en.game24.ru/6726/
109,Game Fire and Water online,http://en.game24.ru/6725/
110,Game online,http://en.game24.ru/6723/
...,...,...
7717,Game Blonde Parking Havoc online,http://en.game24.ru/161/
7718,Game Driving School Parking online,http://en.game24.ru/160/
7719,Game Cinema Drive in Parking online,http://en.game24.ru/159/
7720,Game American Muscle Car Parking online,http://en.game24.ru/156/


In [ ]:
raw_data = []
for i,row in df_games.iterrows():
    if i%100 == 0:
        print("processed", i)
    response = requests.get(row['url'])
    soup = BeautifulSoup(response.text, 'html.parser')
    desc = soup.find(text=re.compile("Description of the game"))
    if desc == "Description of the game  line. How to play the online game  ":
        desc = 'NOT_FOUND'
    
    ratecount = int(soup.find('span', class_='votes').text)
    rate = float(soup.find('span', class_='rating').find('span', class_='average').text)
    playcount = int(soup.find('td', text=re.compile("Played:")).text.split("Played: ")[1].rstrip())
    
    raw_data.append({'game':row['game'],
                     'url':row['url'], 
                     'desc':desc, 
                     'playcount':playcount, 
                     'rating':rate, 
                     'rate_count':ratecount})
df_game_info = pd.DataFrame(raw_data)
df_game_info         

processed 200
processed 300
processed 400
processed 500
processed 600
processed 700
processed 800
processed 900
processed 1000
processed 1100
processed 1200
processed 1300
processed 1400
processed 1500
processed 1600
processed 1700
processed 1800
processed 1900
processed 2000
processed 2100
processed 2200
processed 2300
processed 2400
processed 2500
processed 2600
processed 2700
processed 2800
processed 2900
processed 3000
processed 3100
processed 3200
processed 3300
processed 3400
processed 3500
processed 3600
processed 3700
processed 3800
processed 3900
processed 4000
processed 4100
processed 4200
processed 4300
processed 4400
processed 4500
processed 4600
processed 4700
processed 4800
processed 4900
processed 5000
processed 5100
processed 5200
processed 5300
processed 5400
processed 5500
processed 5600
processed 5700
processed 5800
processed 5900
processed 6000
processed 6100
processed 6200
processed 6300
processed 6400
processed 6500
processed 6600
processed 6700
processed 6800
pro

In [97]:
df_game_info.to_csv("game24ru_game_info.csv") 

### hotgames.kz

In [102]:
response = requests.get("http://hotgames.kz")
menu = BeautifulSoup(response.text, 'html.parser').find('div',id="menu")

cat_urls = [x['href'] for x in menu.find_all('a')]
categories = [x.split('/')[-2] for x in cat_urls]

for x,y in zip(cat_urls, cat):
    print(x,y)

http://hotgames.kz/arcade/ arcade
http://hotgames.kz/race/ race
http://hotgames.kz/logic/ logic
http://hotgames.kz/shooters/ shooters
http://hotgames.kz/platformers/ platformers
http://hotgames.kz/strategy/ strategy
http://hotgames.kz/igry-dlja-devochek/ igry-dlja-devochek


In [110]:
raw_data = []

for cat_url, cat in zip(cat_urls, categories):
    processed = set()
    page_no = 1
    while True:
        print(cat, cat_url + str(page_no), page_no)
        response = requests.get(cat_url + str(page_no))
        soup = BeautifulSoup(response.text, 'html.parser')
        game_containers = soup.find_all('div', class_="game_block")
        curr_games = set()
        
        for container in game_containers:
            game = container.find('div', class_="game_link").text
            url = container.find('div', class_="game_link").find('a')['href']
            desc = container.find('div', class_='game_description').text.lstrip()[:-1]
            rating = container.find('div', class_='game_rating_group').find('img')['alt'].split(': ')[1]
#             print(game)
#             print(url)
#             print(desc)
#             print(rating)
            
            curr_games.add(game)
            raw_data.append({'game':game[1:-1], 
                             'url':url, 
                             'desc':desc, 
                             'category':cat, 
                             'rating':rating})
        #no new games
        if curr_games - processed==set():
            break
        processed = processed | curr_games
        page_no +=1
df_games = pd.DataFrame(raw_data)
df_games

arcade http://hotgames.kz/arcade/1 1
arcade http://hotgames.kz/arcade/2 2
arcade http://hotgames.kz/arcade/3 3
arcade http://hotgames.kz/arcade/4 4
arcade http://hotgames.kz/arcade/5 5
arcade http://hotgames.kz/arcade/6 6
arcade http://hotgames.kz/arcade/7 7
race http://hotgames.kz/race/1 1
race http://hotgames.kz/race/2 2
race http://hotgames.kz/race/3 3
logic http://hotgames.kz/logic/1 1
logic http://hotgames.kz/logic/2 2
logic http://hotgames.kz/logic/3 3
logic http://hotgames.kz/logic/4 4
logic http://hotgames.kz/logic/5 5
logic http://hotgames.kz/logic/6 6
logic http://hotgames.kz/logic/7 7
shooters http://hotgames.kz/shooters/1 1
shooters http://hotgames.kz/shooters/2 2
shooters http://hotgames.kz/shooters/3 3
platformers http://hotgames.kz/platformers/1 1
platformers http://hotgames.kz/platformers/2 2
platformers http://hotgames.kz/platformers/3 3
platformers http://hotgames.kz/platformers/4 4
strategy http://hotgames.kz/strategy/1 1
strategy http://hotgames.kz/strategy/2 2
igry

,game,url,desc,category,rating
0,Сокровища пиратов,http://hotgames.kz/game/sokrovisha-piratov/,«Сокровища пиратов» — онлайн игра для любителе...,arcade,2.9
1,Нян Кэт,http://hotgames.kz/game/nyan-cat/,Очень популярная игра про забавную кошечку по ...,arcade,3.5
2,Пушка пирата,http://hotgames.kz/game/pushka-pirata/,Что делают пираты после долгих плаваний когда ...,arcade,1.9
3,Flaming Zombooka 3: Carnival,http://hotgames.kz/game/flaming-zombooka-3-car...,Зомби клоуны захватили цирк! Вам предстоит в р...,arcade,1.2
4,Ballooner,http://hotgames.kz/game/ballooner/,Физическай игра в которой нужно освободить все...,arcade,2.1
...,...,...,...,...,...
263,Siegius,http://hotgames.kz/game/siegius/,В этой игре вы выступаете в роли Цезаря. Управ...,strategy,3.7
264,Tesla death ray,http://hotgames.kz/game/tesla-death-ray/,"Трансформатор Тесла, одно из изобретений велик...",strategy,3.1
265,Red Storm Defense,http://hotgames.kz/game/red-storm-defense/,Стратегическая игра. Защити свои кристаллы от ...,strategy,3.4
266,Back2Back,http://hotgames.kz/game/back2back/,Тактическая игра в которой необходимо отбивать...,strategy,3.4


In [111]:
df_games.to_csv("hotgameskz_game_info.csv")

### iogamez.net

In [127]:
response = requests.get("https://iogame.app")
menu = BeautifulSoup(response.text, 'html.parser').find('ul',id="menu-menu")



categories = [x.text for x in menu.find_all('li')]
cat_urls = [x.find('a')['href'] for x in menu.find_all('li')]

for x,y in zip(categories, cat_urls):
    print(x,y)

Agar Style https://iogame.app/category/agar-style/
Asteroids https://iogame.app/category/asteroids/
Classic https://iogame.app/category/classic/
Diep Style https://iogame.app/category/diep-style/
Maze https://iogame.app/category/maze/
Moomoo Style https://iogame.app/category/moomoo-style/
Ocean https://iogame.app/category/ocean/
Paper Style https://iogame.app/category/paper-style/
Power-Ups https://iogame.app/category/power-ups/
Racing https://iogame.app/category/racing/
RPG https://iogame.app/category/rpg/
Shooter https://iogame.app/category/shooter/
Snake Style https://iogame.app/category/snake-style/
Space https://iogame.app/category/space/
Sports https://iogame.app/category/sports/
Strategy https://iogame.app/category/strategy/
Tank https://iogame.app/category/tank/
TD https://iogame.app/category/td/
Team https://iogame.app/category/team/
Upgrades https://iogame.app/category/upgrades/
Zombie https://iogame.app/category/zombie/


In [133]:
raw_data = []

for cat_url, cat in zip(cat_urls, categories):

    page_no = 1
    while True:
        print(cat, cat_url + 'page/' + str(page_no))
        response = requests.get(cat_url + 'page/' + str(page_no))
        soup = BeautifulSoup(response.text, 'html.parser')
        game_containers = soup.find_all('div', class_="fusion-post-wrapper")
        if game_containers == []:
            break
        for container in game_containers:
            gameblocks = container.find_all('div', class_='fusion-post-content-wrapper')
            for block in gameblocks:
#                 print(block.find('a')['href'], block.find('a').text)
                raw_data.append({'game':block.find('a').text, 
                                'url':block.find('a')['href'], 
                                'category':cat})
        page_no +=1

df_games = pd.DataFrame(raw_data)
df_games

Agar Style https://iogame.app/category/agar-style/page/1
Agar Style https://iogame.app/category/agar-style/page/2
Asteroids https://iogame.app/category/asteroids/page/1
Asteroids https://iogame.app/category/asteroids/page/2
Classic https://iogame.app/category/classic/page/1
Classic https://iogame.app/category/classic/page/2
Classic https://iogame.app/category/classic/page/3
Diep Style https://iogame.app/category/diep-style/page/1
Diep Style https://iogame.app/category/diep-style/page/2
Maze https://iogame.app/category/maze/page/1
Maze https://iogame.app/category/maze/page/2
Moomoo Style https://iogame.app/category/moomoo-style/page/1
Moomoo Style https://iogame.app/category/moomoo-style/page/2
Ocean https://iogame.app/category/ocean/page/1
Ocean https://iogame.app/category/ocean/page/2
Paper Style https://iogame.app/category/paper-style/page/1
Paper Style https://iogame.app/category/paper-style/page/2
Power-Ups https://iogame.app/category/power-ups/page/1
Power-Ups https://iogame.app/c

,game,url,category
0,towerz.io,https://iogame.app/towerz-io/,Agar Style
1,hury.io,https://iogame.app/hury-io/,Agar Style
2,yumy.io,https://iogame.app/yumy-io/,Agar Style
3,Eat Up 3D,https://iogame.app/eat-up-3d/,Agar Style
4,pie.ai,https://iogame.app/pie-ai/,Agar Style
...,...,...,...
491,zomzom.io,https://iogame.app/zomzom-io/,Zombie
492,wowz.io,https://iogame.app/wowz-io/,Zombie
493,pumking.io,https://iogame.app/pumking-io/,Zombie
494,modd.io,https://iogame.app/modd-io-2/,Zombie


In [134]:
df_games.to_csv("iogameznet_game_info.csv")